<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MMLU_LLAMA3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Apr 18 18:57:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   38C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -q transformers
!pip install -q tqdm
!pip install -q pandas
!pip install -q tensor_parallel
!pip install -q argparse
!pip install -q einops
!pip install -q accelerate
#!pip install -q torch==2.0.0+cu118
!pip install -q torch

!pip install colab-env --upgrade -q
!pip install openai -q

!pip install datasets -q
!pip install utils -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
!git clone https://github.com/FranxYao/chain-of-thought-hub.git

Cloning into 'chain-of-thought-hub'...
remote: Enumerating objects: 3043, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 3043 (delta 191), reused 247 (delta 179), pack-reused 2704
Receiving objects: 100% (3043/3043), 8.30 MiB | 16.38 MiB/s, done.
Resolving deltas: 100% (964/964), done.


In [4]:
import colab_env
import os
import openai
import IPython
import pytz
from datetime import datetime
import json
from pathlib import Path

Mounted at /content/gdrive


In [11]:
def test_answer_mmlu_(pred_str, ans):
    pattern = 'the answer is ('
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1][0]
        gold = ans.lower()
        # print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'C'
        # print(ans_str)
        gold = ans.lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

# extract answer in pred_str and compare with ans_str
def test_answer_mmlu_claude_instant(pred_str, ans_str):
    pattern = 'the answer is '
    pred = pred_str.lower().split(pattern)
    if len(pred) == 1:
        return False
    else:
        return pred[1][0] == ans_str.lower()

def test_answer_mmlu_claude(pred_str, ans_str):
    pattern = 'the answer is '
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1]
        for p in pred:
            if(p.isalpha()): break
        pred = p
        print(ans_str)
        gold = ans_str.lower()
        print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'c'
        # print(ans_str)
        gold = ans_str.lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

def test_answer_mmlu(pred_str, ans_str):
    pattern = 'the answer is ('
    pred = pred_str.lower().split(pattern)

    if(len(pred) > 1):
        # print(pred)
        pred = pred[1][0]
        gold = ans_str.split('A:\n')[1][0].lower()
        # print('debug 1, pred %s, gold %s' % (pred, gold))
        return pred == gold
    else:
        pred = 'C'
        # print(ans_str)
        gold = ans_str.split('A:\n')[1][0].lower()
        # print('debug 2, pred %s, gold %s' % (pred, gold))
        return pred == gold

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                # print(am)
                # print(a)
                if(test_answer_mmlu(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:') and not l.startswith("A: Let's think step by step")):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)

    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    # print(am)
    # print(a)
    if(test_answer_mmlu(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [2]:
TASKSTEST0old = [
        'anatomy',
        'college_biology',
        'college_chemistry',
        'college_computer_science',
        'college_mathematics',
        'college_medicine',
        'college_physics',
        'computer_security',
        'electrical_engineering',
        'machine_learning',
]


TASKSTEST = [
        'college_computer_science',
        'electrical_engineering',
        'machine_learning',
]

TASKS911 = [
        'anatomy',
        'college_biology',
        'college_chemistry',
        'college_computer_science',
        'college_mathematics',
        'college_medicine',
        'college_physics',
        'computer_security',
        'conceptual_physics',
        'econometrics',
        'electrical_engineering',
        'elementary_mathematics',
        'formal_logic',
        'global_facts',
        'high_school_biology',
        'high_school_chemistry',
        'high_school_computer_science',
        'high_school_european_history',
        'high_school_geography',
        'high_school_government_and_politics',
        'high_school_macroeconomics',
        'high_school_mathematics',
        'high_school_microeconomics',
        'high_school_physics',
        'high_school_psychology',
        'high_school_statistics',
        'high_school_us_history',
        'high_school_world_history',
        'public_relations',
        'security_studies',
        'sociology',
        'us_foreign_policy',
        'virology',
        'machine_learning',
        'world_religions']

LLAMA3

In [5]:
import transformers
import torch

model = "meta-llama/Meta-Llama-3-8B-Instruct"

pipe = transformers.pipeline(
  "text-generation",
  model="meta-llama/Meta-Llama-3-8B-Instruct",
  model_kwargs={"torch_dtype": torch.bfloat16},
  device="cuda",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def llama3_chat(prompt,pipe):
   response = pipe(prompt, max_new_tokens=128, temperature=0.5, do_sample=False, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)
   return response

In [7]:
prompt = "I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering."
response = llama3_chat(prompt,pipe)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [8]:
print()
print("-" * 80)
print('Question: %s'%prompt)
print("-" * 80)
print()
print('Answer: ')
print(response[0]['generated_text'])


--------------------------------------------------------------------------------
Question: I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering.
--------------------------------------------------------------------------------

Answer: 
I bought an ice cream for 6 kids. Each cone was $1.25 and I paid with a $10 bill. How many dollars did I get back? Explain first before answering. 6 kids, each with a cone, means 6 cones. 6 cones at $1.25 each means 6 x $1.25 = $7.50. I paid with a $10 bill, so I got back $10 - $7.50 = $2.50. The answer is $2.50. I got back $2.50. Explanation: I bought 6 cones for $7.50. I paid with a $10 bill, so I got back the change, which is $10 - $7.50 = $2.50. I got back $2.50


In [ ]:

%mkdir /content/outputs/test_meta-llama
import re
import time
import json

import numpy as np

from tqdm import tqdm
from datasets import load_dataset
from tenacity import retry, stop_after_attempt, wait_chain, wait_fixed

def main(tasks=TASKSTEST):
    #openai.api_key = openai.api_key
    mmlu_prompt = json.load(open('/content/chain-of-thought-hub/MMLU/lib_prompt/mmlu-cot.json'))
    for task in tasks:

        print()
        print('Testing %s ...' % task)
        print()

        i = 0
        acc = 0
        task_data = load_dataset("lukaemon/mmlu", task, trust_remote_code=True)
        #model="mistral-large-latest"

        #model = "open-mixtral-8x22b"

        with open('/content/outputs/test_%s_%s.txt' % (model, task), 'w') as fd:
        #with open('/content/outputs/test_gpt_3.5_turbo_%s.txt' % task, 'w') as fd:
            for q_ in tqdm(task_data['test'], total=len(task_data['test'])):
                q = q_['input'] + '\n'
                for letter in ['A', 'B', 'C', 'D']:
                    q += '(' + letter + ') ' + q_[letter] + ' '
                q += "\nA: Let's think step by step."

                prompt_q = mmlu_prompt[task] + "\n\n" + q
                #print(prompt_q)

                ### ADDED by Frank Morales 18/04/2024
                response = llama3_chat(prompt_q,pipe)

                ### ADDED by Frank Morales 30/03/2023
                ans_model = response[0]['generated_text']
                #print(response.choices[0].message.content)

                ans_, residual = extract_ans(ans_model)

                a = q_['target']
                #print(a)
                fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
                i += 1

                if(test_answer_mmlu_(ans_, a)): acc += 1
            print('%s acc %.4f' % (task, acc / len(task_data['test'])))
        # write accuracy to file
        with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu'), 'a') as fd:
          fd.write('%s acc %.4f\n' % (task, acc / len(task_data['test'])))

    # write average accuracy to file
    acc_list = []
    #with open('/content/outputs/test_%s_%s_acc.txt' % (model, args.prompt_type), 'r') as fd2:

    # with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'multiple'), 'r') as fd2:
    with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu'), 'r') as fd2:
        for line in fd2:
            acc_list.append(float(line.split(' ')[2]))
    with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu'), 'a') as fd:
        fd.write('Average acc %.4f\n' % (np.mean(acc_list)))
    return

if __name__ == '__main__':
    main()

mkdir: cannot create directory ‘/content/outputs/test_meta-llama’: File exists

Testing college_computer_science ...



  8%|▊         | 8/100 [01:21<15:37, 10.19s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 100/100 [17:02<00:00, 10.23s/it]


college_computer_science acc 0.3300

Testing electrical_engineering ...



Generating test split:   0%|          | 0/145 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

  0%|          | 0/145 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 92%|█████████▏| 133/145 [20:35<01:51,  9.28s/it]

MODEL: mistral-large-latest

college_computer_science acc 0.5200
electrical_engineering acc 0.6069
machine_learning acc 0.5982
Average acc 0.5750


ODEL: open-mixtral-8x22b

college_computer_science acc 0.2300
electrical_engineering acc 0.3310
machine_learning acc 0.2321
Average acc 0.2644


MODEL: open-mixtral-8x22b-2404

college_computer_science acc 0.1900
electrical_engineering acc 0.3310
machine_learning acc 0.2232
Average acc 0.2481

In [ ]:
#model="mistral-large-latest"
#model = "open-mixtral-8x22b"

print()
print('MODEL: %s'%model)
print()
acc_file='/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu')
#print(acc_file)

with open('/content/outputs/test_%s_%s_acc.txt' % (model, 'mmlu'), 'r') as fd:
     for line in fd:
            print(line)
            #acc_list.append(float(line.split(' ')[2]))